In [50]:
from dotenv import load_dotenv
import os
import requests
load_dotenv('../.env')

True

In [51]:
BEARER_TOKEN = os.getenv('GITHUB_TOKEN')
if BEARER_TOKEN :
    print('GITHUB TOKEN found in envirionment')
else:
   print('GITHUB TOKEN not found')

GITHUB TOKEN found in envirionment


In [ ]:
USER = 'Kaggle' # will take input from the user instead hardcoded
BASE_URL = 'https://api.github.com/'


response = requests.get(
    f'{BASE_URL}/users/{USER}/repos',
    headers={
        'Authorization': f'Bearer {BEARER_TOKEN}'
    }
)
repositories = response.json()
repositories

In [53]:
class Repository:
    name = ''
    stargazers_count = 0
    forks_count = 0

    # further api metrics

    commits_count = 0
    contributors_count = 0
    branches_count = 0
    tags_count = 0
    issues_count = 0
    releases_count = 0
    closed_issues_count = 0
    enviroments_count = 0

    def __init__(self, name, stargazers_count, forks_count):
        self.name = name
        self.stargazers_count = stargazers_count
        self.forks_count = forks_count

        self.commits_count = self.__get_metrics_count('commits')
        self.contributors_count = self.__get_metrics_count('contributors')
        self.branches_count = self.__get_metrics_count('branches')
        self.tags_count = self.__get_metrics_count('tags')
        self.issues_count = self.__get_metrics_count('issues')
        self.releases_count = self.__get_metrics_count('releases')        
  
    def __get_metrics_count(self, url_extension):
        url = f'{BASE_URL}/repos/{USER}/{self.name}/{url_extension}'
        response = requests.get(url, headers={
            'Authorization': f'Bearer {BEARER_TOKEN}'
        })
        return len(response.json())
    
    def to_dict(self):
        return {
            'name': self.name,
            'stargazers_count': self.stargazers_count,
            'forks_count': self.forks_count,
            'commits_count': self.commits_count,
            'contributors_count': self.contributors_count,
            'branches_count': self.branches_count,
            'tags_count': self.tags_count,
            'issues_count': self.issues_count,
            'releases_count': self.releases_count,
        }



In [54]:
print(repositories[0]['name'])

.allstar


In [57]:
repo = Repository(repositories[1]['name'], repositories[1]['stargazers_count'], repositories[1]['forks_count'])
repo.to_dict()


{'name': 'docker-julia',
 'stargazers_count': 36,
 'forks_count': 12,
 'commits_count': 30,
 'contributors_count': 6,
 'branches_count': 1,
 'tags_count': 0,
 'issues_count': 1,
 'releases_count': 0}

In [56]:
repo_metrics = []
for repo in repositories:
    repo_metrics.append(Repository(repo['name'], repo['stargazers_count'], repo['forks_count']).to_dict())
repo_metrics


[{'name': '.allstar',
  'stargazers_count': 1,
  'forks_count': 0,
  'commits_count': 1,
  'contributors_count': 0,
  'branches_count': 1,
  'tags_count': 0,
  'issues_count': 0,
  'releases_count': 0},
 {'name': 'docker-julia',
  'stargazers_count': 36,
  'forks_count': 12,
  'commits_count': 30,
  'contributors_count': 6,
  'branches_count': 1,
  'tags_count': 0,
  'issues_count': 1,
  'releases_count': 0},
 {'name': 'docker-python',
  'stargazers_count': 2566,
  'forks_count': 980,
  'commits_count': 30,
  'contributors_count': 30,
  'branches_count': 30,
  'tags_count': 30,
  'issues_count': 28,
  'releases_count': 30},
 {'name': 'docker-rcran',
  'stargazers_count': 20,
  'forks_count': 14,
  'commits_count': 30,
  'contributors_count': 9,
  'branches_count': 8,
  'tags_count': 0,
  'issues_count': 4,
  'releases_count': 0},
 {'name': 'docker-rstats',
  'stargazers_count': 144,
  'forks_count': 91,
  'commits_count': 30,
  'contributors_count': 18,
  'branches_count': 4,
  'tags_c